<a href="https://colab.research.google.com/github/tanveerkgill/nhanes_inferential_2021_23/blob/main/nhanes_inferential_2021_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Show all columns when viewing dataframes
pd.set_option("display.max_columns", None)


In [9]:
# Function to load XPT files
def read_xpt_file(file_path):
    return pd.read_sas(file_path, format="xport")

# Load the datasets
demo_df = read_xpt_file("sample_data/DEMO_L.xpt")
bp_df   = read_xpt_file("sample_data/BPXO_L.xpt")
vit_df  = read_xpt_file("sample_data/VID_L.xpt")
hep_df  = read_xpt_file("sample_data/HEPB_S_L.xpt")
kid_df  = read_xpt_file("sample_data/KIQ_U_L.xpt")
paq_df  = read_xpt_file("sample_data/PAQ_L.xpt")
wh_df   = read_xpt_file("sample_data/WHQ_L.xpt")

demo_df.head()


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMDBORN4,DMDYRUSR,DMDEDUC2,DMDMARTZ,RIDEXPRG,DMDHHSIZ,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
0,130378.0,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,2.0,2.0,6.0,5.0,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379.0,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,2.0,1.0,NaN,5.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380.0,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,2.0,2.0,6.0,3.0,1.0,2.0,7.0,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130381.0,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,NaN,1.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,3.0,NaN,38807.268902,55698.607106,182.0,2.0,1.53
4,130382.0,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,NaN,1.0,NaN,NaN,NaN,NaN,4.0,2.0,2.0,3.0,1.0,2.0,30607.519774,36434.146346,182.0,2.0,3.60


In [10]:
# Adults only for demographic questions (20+)
adult_demo = demo_df.loc[demo_df["RIDAGEYR"] >= 20].copy()

# Recode marital status (1 = Married/Living with partner)
adult_demo["marital_status"] = (adult_demo["DMDMARTZ"] == 1).astype(int)

# Recode education (5 = Bachelor's or above)
adult_demo["edu_bachelors_plus"] = (adult_demo["DMDEDUC2"] == 5).astype(int)

# Clean sedentary minutes (replace placeholders with NaN)
paq_df["PAD680"] = paq_df["PAD680"].replace([7777, 9999], np.nan)

# Clean self-reported weight
wh_df["WHD020"] = wh_df["WHD020"].replace([7777, 9999], np.nan)

# Quick frequency checks
print("Marital status recode:")
print(adult_demo["marital_status"].value_counts(), "\n")

print("Education recode:")
print(adult_demo["edu_bachelors_plus"].value_counts(), "\n")

print("PAD680 cleaned check:")
print(paq_df["PAD680"].value_counts(dropna=False).head(), "\n")

print("WHD020 cleaned check:")
print(wh_df["WHD020"].value_counts(dropna=False).head())


Marital status recode:
marital_status
1    4136
0    3673
Name: count, dtype: int64 

Education recode:
edu_bachelors_plus
0    5184
1    2625
Name: count, dtype: int64 

PAD680 cleaned check:
PAD680
240.0    1130
360.0     983
180.0     905
300.0     905
480.0     896
Name: count, dtype: int64 

WHD020 cleaned check:
WHD020
160.0    308
180.0    299
150.0    273
170.0    273
140.0    251
Name: count, dtype: int64


In [11]:
# Q1: Relationship between marital status and education level

q1_data = adult_demo[["marital_status", "edu_bachelors_plus"]].dropna()

# Crosstab
cross_tab_q1 = pd.crosstab(q1_data["marital_status"], q1_data["edu_bachelors_plus"])
print("Contingency Table:\n")
print(cross_tab_q1, "\n")

# Chi-square test
chi2_val, p_val, df_val, exp = st.chi2_contingency(cross_tab_q1)

print(f"Chi-square: {chi2_val:.3f}")
print(f"Degrees of freedom: {df_val}")
print(f"p-value: {p_val:.4f}")

print("""
Interpretation:
A statistically significant p-value (< 0.05) indicates that marital
status and education level are associated in this dataset.
""")


Contingency Table:

edu_bachelors_plus     0     1
marital_status                
0                   2679   994
1                   2505  1631 

Chi-square: 132.883
Degrees of freedom: 1
p-value: 0.0000

Interpretation:
A statistically significant p-value (< 0.05) indicates that marital 
status and education level are associated in this dataset.



In [12]:
# Q2: Compare means of sedentary behavior between married vs not married

q2_merge = (
    adult_demo[["SEQN", "marital_status"]]
    .merge(paq_df[["SEQN", "PAD680"]], on="SEQN", how="inner")
    .dropna(subset=["PAD680"])
)

married_group     = q2_merge.loc[q2_merge["marital_status"] == 1, "PAD680"]
not_married_group = q2_merge.loc[q2_merge["marital_status"] == 0, "PAD680"]

print("Mean sedentary minutes — married:", married_group.mean())
print("Mean sedentary minutes — not married:", not_married_group.mean(), "\n")

t_stat, p_val = st.ttest_ind(married_group, not_married_group, equal_var=False)

print(f"T-statistic: {t_stat:.2f}")
print(f"p-value: {p_val:.4f}")

print("""
Interpretation:
If p < 0.05, there is a meaningful difference in average sedentary
minutes/day between married and unmarried adults.
""")


Mean sedentary minutes — married: 353.28714076960546
Mean sedentary minutes — not married: 371.6780878695772 

T-statistic: -3.80
p-value: 0.0001

Interpretation:
If p < 0.05, there is a meaningful difference in average sedentary
minutes/day between married and unmarried adults.



In [13]:
# Q3: Examine how age and marital status relate to systolic BP

bp_merge = (
    adult_demo[["SEQN", "RIDAGEYR", "marital_status", "DMDMARTZ"]]
    .merge(bp_df[["SEQN", "BPXOSY3"]], on="SEQN", how="inner")
    .dropna(subset=["BPXOSY3", "RIDAGEYR"])
)

# Label for categorical variable
bp_merge["marital_label"] = bp_merge["marital_status"].map({1: "Married", 0: "Not_married"})

# ANOVA model
anova_model = smf.ols("BPXOSY3 ~ C(marital_label) * RIDAGEYR", data=bp_merge).fit()
anova_results = sm.stats.anova_lm(anova_model, typ=2)

print(anova_results)

print("""
Interpretation:
Age typically shows a strong influence on systolic BP.
Marital status may have a smaller effect, and the interaction term
reveals whether age impacts BP differently by marital group.
""")


                                 sum_sq      df           F         PR(>F)
C(marital_label)           2.594421e+03     1.0    8.872488   2.906911e-03
RIDAGEYR                   2.655028e+05     1.0  907.975184  1.566450e-185
C(marital_label):RIDAGEYR  1.842506e+02     1.0    0.630106   4.273486e-01
Residual                   1.707393e+06  5839.0         NaN            NaN

Interpretation:
Age typically shows a strong influence on systolic BP.
Marital status may have a smaller effect, and the interaction term
reveals whether age impacts BP differently by marital group.



In [14]:
# Q4: Is weight related to sedentary behavior?

q4_join = (
    wh_df[["SEQN", "WHD020"]]
    .merge(paq_df[["SEQN", "PAD680"]], on="SEQN", how="inner")
    .dropna(subset=["WHD020", "PAD680"])
)

# Restrict to realistic human ranges
q4_filtered = q4_join[
    q4_join["WHD020"].between(30, 250) &
    q4_join["PAD680"].between(0, 1440)
]

# Spearman correlation
rho, p_corr = spearmanr(q4_filtered["WHD020"], q4_filtered["PAD680"])

print(f"Spearman rho: {rho:.2f}")
print(f"p-value: {p_corr:.4f}")

print("""
Interpretation:
A small positive rho means higher weight is weakly associated with
slightly more sedentary time — but the correlation is weak.
""")


Spearman rho: 0.11
p-value: 0.0000

Interpretation:
A small positive rho means higher weight is weakly associated with
slightly more sedentary time — but the correlation is weak.



In [15]:
# Q5: Are adults with weak/failing kidneys older than those without?

q5 = (
    demo_df[["SEQN", "RIDAGEYR"]]
    .merge(kid_df[["SEQN", "KIQ022"]], on="SEQN", how="inner")
    .dropna(subset=["RIDAGEYR", "KIQ022"])
)

# Only keep valid two-level responses
q5 = q5[q5["KIQ022"].isin([1, 2])]

age_yes = q5.loc[q5["KIQ022"] == 1, "RIDAGEYR"]
age_no  = q5.loc[q5["KIQ022"] == 2, "RIDAGEYR"]

print("Mean age (kidney issue = yes):", age_yes.mean())
print("Mean age (kidney issue = no):", age_no.mean(), "\n")

t_res = st.ttest_ind(age_yes, age_no, equal_var=False)
print(f"T-statistic: {t_res.statistic:.2f}")
print(f"p-value: {t_res.pvalue:.4f}")

print("""
Interpretation:
If the p-value is below 0.05, adults who report weak or failing kidneys
are significantly older on average.
""")


Mean age (kidney issue = yes): 65.81308411214954
Mean age (kidney issue = no): 53.076274588518665 

T-statistic: 16.01
p-value: 0.0000

Interpretation:
If the p-value is below 0.05, adults who report weak or failing kidneys 
are significantly older on average.

